# Ecart stationnaire à la moyenne zonale

**Auteur : FERRY Frédéric (DESR/ENM/C3M) - septembre 2021

Les fichiers de données au format netcdf (moyennes mensuelles NCEP) doivent être récupérés par FTP (FileZilla) et placés dans le répertoire data :

Hauteur géopotentielle, vent zonal, vent méridien :
- ftp://ftp.cdc.noaa.gov/Datasets/ncep.reanalysis.derived/pressure/hgt.mon.mean.nc
- ftp://ftp.cdc.noaa.gov/Datasets/ncep.reanalysis.derived/pressure/uwnd.mon.mean.nc
- ftp://ftp.cdc.noaa.gov/Datasets/ncep.reanalysis.derived/pressure/vwnd.mon.mean.nc

In [ ]:
%matplotlib inline

import os

import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
import matplotlib.path as mpath

from cartopy.util import add_cyclic_point
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

import IPython.display as IPdisplay, matplotlib.font_manager as fm
from PIL import Image
import glob

from scipy.integrate import cumtrapz

import warnings
warnings.filterwarnings('ignore')

# Traitement des données

In [ ]:
diri="./data/"
fz    = xr.open_dataset(diri+"hgt.mon.mean.nc")
fu    = xr.open_dataset(diri+"uwnd.mon.mean.nc")
fv    = xr.open_dataset(diri+"vwnd.mon.mean.nc")
print(fz)
print(fu)
print(fv)

In [ ]:
year1=input("année de départ : ")
year2=input("année de fin : ")

In [ ]:
fz    = xr.open_dataset(diri+"hgt.mon.mean.nc").sel(time=slice(year1,year2))
fu    = xr.open_dataset(diri+"uwnd.mon.mean.nc").sel(time=slice(year1,year2))
fv    = xr.open_dataset(diri+"vwnd.mon.mean.nc").sel(time=slice(year1,year2))

lat  = fz.lat.values
lev = fz.level.values

print(fz)
print(lev)

In [ ]:
seasons=['DJF','JJA','MAM','SON']

fz_mean = fz.groupby('time.season').mean('time')
fu_mean = fu.groupby('time.season').mean('time')
fv_mean = fv.groupby('time.season').mean('time')

fz_zmean = fz_mean.mean('lon')
z0, z0_zmean, = xr.broadcast(fz_mean['hgt'],fz_zmean['hgt'])
z0_anom=z0-z0_zmean # ecart a la moyenne zonale du géopotentiel
u0 = fu_mean['uwnd']
v0 = fv_mean['vwnd']

print(z0.shape)

In [ ]:
def lonflip(da):
    lon_name = 'lon'
    da['_longitude_adjusted'] = xr.where(
        da[lon_name] > 180,
        da[ lon_name] - 360,
        da[lon_name])
    da = (
        da
        .swap_dims({lon_name: '_longitude_adjusted'})
        .sel(**{'_longitude_adjusted': sorted(da._longitude_adjusted)})
        .drop(lon_name))
    da = da.rename({'_longitude_adjusted': lon_name})
    return da

In [ ]:
z=lonflip(z0)
z_anom=lonflip(z0_anom)
uu=lonflip(u0)
vv=lonflip(v0)

lon  = z.lon.values
print(lon)

# Conversion de u et v en numpy array pour tracé vecteurs
u=np.array(uu)
v=np.array(vv)

# Fonctions graphiques

In [ ]:
#proj=ccrs.PlateCarree()
proj=ccrs.EqualEarth()

def plot_background(ax):
    ax.coastlines()
    ax.gridlines(crs=ccrs.PlateCarree(), linewidth=0.5, color='gray', alpha=0.5, linestyle='-')
    return ax

In [ ]:
def plot_zonal(ax):
    ax.set_yscale('symlog')
    ax.set_yticklabels(np.arange(1000, 0, -100))
    ax.set_ylim(1000, 100)
    ax.set_yticks(np.arange(1000, 0, -100))  
    ax.set_xticklabels(np.arange(-180, 190, 10))
    ax.set_xticks(np.arange(-180, 190, 10)) 
    return ax

# Ecart à la moyenne zonale du géopotentiel à 200hPa

In [ ]:
plevz=200
ilev = list(lev).index(plevz)
levels_z_anom = np.arange(-200,220,20)


axes_class = (GeoAxes, dict(map_projection=proj))
fig = plt.figure(figsize=(15,15))
fig.suptitle('Geopotential height (mgp) at '+str(plevz)+' hPa - anomaly from zonal mean : NCEP '+year1+'-'+year2, fontsize=16)

axgr = AxesGrid(fig, 111, axes_class=axes_class,
       nrows_ncols=(2, 1),
       axes_pad=0.6,
       cbar_location='right',
       cbar_mode='single',
       cbar_pad=0.2,
       cbar_size='3%',
       label_mode='')  # note the empty label_mode
                   
for i, ax in enumerate(axgr):
   ax.set_title(seasons[i], fontsize=14)
   plot_background(ax)
   cf = ax.contourf(lon, lat, z_anom[i,ilev,:,:], levels_z_anom, transform=ccrs.PlateCarree(), cmap='coolwarm', extend='both')
   c = ax.contour(lon, lat, z_anom[i,ilev,:,:], levels_z_anom, colors='black', linewidths=0.1, transform=ccrs.PlateCarree())
   axgr.cbar_axes[i].colorbar(cf)

figname='./figs/z'+str(plevz)+'_anomaly_global_climatology'
fig.savefig(figname+'.png',bbox_inches='tight')

plt.show()

# Ecart à la moyenne zonale du géopotentiel - coupe verticale

In [ ]:
levels_z_anom = np.arange(-600,650,50)

latN=60
latS=-60

print(z_anom.shape)
ilatN = list(lat).index(latN)
ilatS = list(lat).index(latS)

fig = plt.figure(figsize=(12, 12))
fig.suptitle('Geopotential height (mgp) - anomaly from zonal mean : NCEP '+year1+'-'+year2, fontsize=16)

ax=fig.add_subplot(211)
ax.set_title('DJF - lat='+str(latN), fontsize=14)
plot_zonal(ax)
cf = ax.contourf(lon, lev, z_anom[0,:,ilatN,:], levels_z_anom, cmap='coolwarm', extend='both')
c = ax.contour(lon, lev, z_anom[0,:,ilatN,:], levels_z_anom, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('Km/s', size='small')

ax=fig.add_subplot(212)
ax.set_title('JJA - lat='+str(latS), fontsize=14)
plot_zonal(ax)
cf = ax.contourf(lon, lev, z_anom[1,:,ilatS,:], levels_z_anom, cmap='coolwarm', extend='both')
c = ax.contour(lon, lev, z_anom[1,:,ilatS,:], levels_z_anom, colors='black', linewidths=1)
#plt.clabel(c, levels_tz, fmt='%1.2i')
cb = fig.colorbar(cf, orientation='horizontal')
cb.set_label('Km/s', size='small')

figname='./figs/z_anomaly_crossection_climatology'
fig.savefig(figname+'.png',bbox_inches='tight')
plt.show()

# Décomposition du vent horizontal en fonction de courant et potentiel de vitesse

<div class="alert alert-danger">
<b>MAC LINUX SEULEMENT (module windspharm)</b>

</div>

In [ ]:
plev_sfvp=200
ilev = list(lev).index(plev_sfvp)

lons = fu_mean.coords['lon']
lon_idx = u0.dims.index('lon')
u2, lons2 = add_cyclic_point(u0.values, coord=lons, axis=lon_idx)
v2, lons2 = add_cyclic_point(v0.values, coord=lons, axis=lon_idx)

print("Original shape -", u0.shape)
print("New shape -", u2.shape)

uu=u2[:,ilev,:,:]
vv=v2[:,ilev,:,:]

In [ ]:
from windspharm.standard import VectorWind
from windspharm.tools import prep_data, recover_data, order_latdim
from windspharm.examples import example_data_path

uwnd, uwnd_info = prep_data(uu, 'tyx')
vwnd, vwnd_info = prep_data(vv, 'tyx')

lats, uwnd, vwnd = order_latdim(lat, uwnd, vwnd)

w = VectorWind(uwnd, vwnd)

sf, vp = w.sfvp()
uchi, vchi = w.irrotationalcomponent()
upsi, vpsi = w.nondivergentcomponent()

sf = recover_data(sf, uwnd_info)
vp = recover_data(vp, uwnd_info)

uchi = recover_data(uchi, uwnd_info)
upsi = recover_data(upsi, uwnd_info)
vchi = recover_data(vchi, uwnd_info)
vpsi = recover_data(vpsi, uwnd_info)

print(sf.shape)
print(vp.shape)

In [ ]:
sf_zmean = sf.mean(axis=2)
print(sf_zmean.shape)
sf_zmean2=sf_zmean[:,:,np.newaxis]
print(sf_zmean2.shape)
sf_anom=sf-sf_zmean2 # ecart a la moyenne zonale de la fonction de courant
print(sf_anom.shape)

In [ ]:
levels_sf = np.arange(-150,160,10)

axes_class = (GeoAxes, dict(map_projection=ccrs.PlateCarree()))
fig = plt.figure(figsize=(15,8))
fig.suptitle('Streamfunction ($10^6$m$^2$s$^{-1}$) at '+str(plev_sfvp)+' hPa : NCEP '+year1+'-'+year2, fontsize=16)

axgr = AxesGrid(fig, 111, axes_class=axes_class,
       nrows_ncols=(2, 2),
       axes_pad=0.6,
       cbar_location='right',
       cbar_mode='single',
       cbar_pad=0.2,
       cbar_size='3%',
       label_mode='')  # note the empty label_mode

for i, ax in enumerate(axgr):
   ax.set_xticks(np.linspace(-180, 180, 13), crs=ccrs.PlateCarree())
   ax.set_yticks(np.linspace(-90, 90, 13), crs=ccrs.PlateCarree())
   ax.axes.axis('tight')
   lon_formatter = LongitudeFormatter(zero_direction_label=True)
   lat_formatter = LatitudeFormatter()
   ax.xaxis.set_major_formatter(lon_formatter)
   ax.yaxis.set_major_formatter(lat_formatter)
   ax.coastlines()
   ax.set_title(seasons[i], fontsize=14)
   cf = ax.contourf(lons2, lat, sf[i,:,:] * 1e-06, levels_sf, transform=ccrs.PlateCarree(),
                    cmap='jet', extend='both')
   c = ax.contour(lons2, lat, sf[i,:,:] * 1e-06, levels_sf, colors='black',
                  linewidths=0.5, transform=ccrs.PlateCarree())    
   axgr.cbar_axes[i].colorbar(cf)
    
figname='./figs/sf200_climatology'
fig.savefig(figname+'.png',bbox_inches='tight')

plt.show()

In [ ]:
levels_sf_anom = np.arange(-30,35,5)

axes_class = (GeoAxes, dict(map_projection=ccrs.PlateCarree()))
fig = plt.figure(figsize=(15,8))
fig.suptitle('Deviation from zonal mean of the streamfunction ($10^6$m$^2$s$^{-1}$) at '+str(plev_sfvp)+' hPa : NCEP '+year1+'-'+year2
             , fontsize=16)

axgr = AxesGrid(fig, 111, axes_class=axes_class,
       nrows_ncols=(2, 2),
       axes_pad=0.6,
       cbar_location='right',
       cbar_mode='single',
       cbar_pad=0.2,
       cbar_size='3%',
       label_mode='')  # note the empty label_mode

for i, ax in enumerate(axgr):
   ax.set_xticks(np.linspace(-180, 180, 13), crs=ccrs.PlateCarree())
   ax.set_yticks(np.linspace(-90, 90, 13), crs=ccrs.PlateCarree())
   ax.axes.axis('tight')
   lon_formatter = LongitudeFormatter(zero_direction_label=True)
   lat_formatter = LatitudeFormatter()
   ax.xaxis.set_major_formatter(lon_formatter)
   ax.yaxis.set_major_formatter(lat_formatter)
   ax.coastlines()
   ax.set_title(seasons[i], fontsize=14)
   cf = ax.contourf(lons2, lat, sf_anom[i,:,:] * 1e-06, levels=levels_sf_anom, transform=ccrs.PlateCarree(),
                    cmap='coolwarm', extend='both')
   c = ax.contour(lons2, lat, sf_anom[i,:,:] * 1e-06, levels=levels_sf_anom, colors='black',
                  linewidths=0.5, transform=ccrs.PlateCarree())    
   axgr.cbar_axes[i].colorbar(cf)

plt.show()

figname='./figs/sf200_anom_climatology'
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
def polar_circle(ax):
    angle = np.linspace(0, 2*np.pi, 100)
    center, radius = [0.5, 0.5], 0.5
    verts = np.vstack([np.sin(angle), np.cos(angle)]).T
    circle = mpath.Path(verts * radius + center)
    ax.set_boundary(circle, transform=ax.transAxes)
    return ax

projection=ccrs.NorthPolarStereo()
bounds_NP = [(-180., 180., 0., 90.)]

axes_class = (GeoAxes, dict(map_projection=projection))
fig = plt.figure(figsize=(15,8))
fig.suptitle('Deviation from zonal mean of the streamfunction ($10^6$m$^2$s$^{-1}$) at '+str(plev_sfvp)+' hPa : NCEP '+year1+'-'+year2
             , fontsize=16)

axgr = AxesGrid(fig, 111, axes_class=axes_class,
       nrows_ncols=(1, 2),
       axes_pad=0.6,
       cbar_location='right',
       cbar_mode='single',
       cbar_pad=0.2,
       cbar_size='3%',
       label_mode='')  # note the empty label_mode
                   
for i, ax in enumerate(axgr):
    ax.set_title(seasons[i], fontsize=14)
    ax.coastlines()
    polar_circle(ax)
    ax.set_extent(*bounds_NP, crs=ccrs.PlateCarree())
    ax.gridlines(crs=ccrs.PlateCarree(), linewidth=0.5, color='gray', alpha=0.5, linestyle='-')
    cf = ax.contourf(lons2, lat, sf_anom[i,:,:] * 1e-06, levels=levels_sf_anom, transform=ccrs.PlateCarree(),
                     cmap='coolwarm', extend='both')
    c = ax.contour(lons2, lat, sf_anom[i,:,:] * 1e-06, levels=levels_sf_anom, colors='black',
                   linewidths=0.5, transform=ccrs.PlateCarree())  
    axgr.cbar_axes[0].colorbar(cf)

figname='./figs/sf200_anom_NH_climatology'
fig.savefig(figname+'.png',bbox_inches='tight')

plt.show()

In [ ]:
projection=ccrs.SouthPolarStereo()
bounds_NP = [(-180., 180., -90., 0.)] 

axes_class = (GeoAxes, dict(map_projection=projection))
fig = plt.figure(figsize=(15,8))
fig.suptitle('Deviation from zonal mean of the streamfunction ($10^6$m$^2$s$^{-1}$) at '+str(plev_sfvp)+' hPa : NCEP '+year1+'-'+year2
             , fontsize=16)

axgr = AxesGrid(fig, 111, axes_class=axes_class,
       nrows_ncols=(1, 2),
       axes_pad=0.6,
       cbar_location='right',
       cbar_mode='single',
       cbar_pad=0.2,
       cbar_size='3%',
       label_mode='')  # note the empty label_mode
                   
for i, ax in enumerate(axgr):
    ax.set_title(seasons[i], fontsize=14)
    ax.coastlines()
    polar_circle(ax)
    ax.set_extent(*bounds_NP, crs=ccrs.PlateCarree())
    ax.gridlines(crs=ccrs.PlateCarree(), linewidth=0.5, color='gray', alpha=0.5, linestyle='-')
    cf = ax.contourf(lons2, lat, sf_anom[i,:,:] * 1e-06, levels=levels_sf_anom, transform=ccrs.PlateCarree(),
                     cmap='coolwarm', extend='both')
    c = ax.contour(lons2, lat, sf_anom[i,:,:] * 1e-06, levels=levels_sf_anom, colors='black',
                   linewidths=0.5, transform=ccrs.PlateCarree())  
    axgr.cbar_axes[0].colorbar(cf)

figname='./figs/sf200_anom_SH_climatology'
fig.savefig(figname+'.png',bbox_inches='tight')

plt.show()